In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [770 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports In

In [2]:
from pyspark.sql import SparkSession
!wget https://jdbc.postgresql.org/download/postgresql-42.2.25.jre7.jar
spark = SparkSession.builder.appName('segment1').config('spark.driver.extraClassPath', '/content/postgresql-42.2.25.jre7.jar').config('spark.driver.memory', '100g').getOrCreate()

--2022-05-17 01:39:28--  https://jdbc.postgresql.org/download/postgresql-42.2.25.jre7.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984862 (962K) [application/java-archive]
Saving to: ‘postgresql-42.2.25.jre7.jar’

postgresql-42.2.25. 100%[===================>] 961.78K  1.20MB/s    in 0.8s    

2022-05-17 01:39:30 (1.20 MB/s) - ‘postgresql-42.2.25.jre7.jar’ saved [984862/984862]



In [ ]:
from pyspark import SparkFiles
url = 'https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseof_all.csv'
spark.sparkContext.addFile(url)
pseof_df = spark.read.csv(SparkFiles.get('pseof_all.csv'), sep=',', header=True)
url = 'https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseoe_all.csv'
spark.sparkContext.addFile(url)
pseoe_df = spark.read.csv(SparkFiles.get('pseoe_all.csv'), sep=',', header=True)

In [ ]:
pseof_df.show(20)
pseoe_df.show(20)

+--------------+----------+-----------+------------+---------+-------+-----------+-----------------+---------+---------+---------+--------+------------+--------------------+------------+--------------------+-------------+---------------------+------------+------------+-------------+-------------------+---------------------------+-------------------+---------------------------+--------------------+----------------------------+-------------------+-------------------+--------------------+
|agg_level_pseo|inst_level|institution|degree_level|cip_level|cipcode|grad_cohort|grad_cohort_years|geo_level|geography|ind_level|industry|y1_grads_emp|y1_grads_emp_instate|y5_grads_emp|y5_grads_emp_instate|y10_grads_emp|y10_grads_emp_instate|y1_grads_nme|y5_grads_nme|y10_grads_nme|status_y1_grads_emp|status_y1_grads_emp_instate|status_y5_grads_emp|status_y5_grads_emp_instate|status_y10_grads_emp|status_y10_grads_emp_instate|status_y1_grads_nme|status_y5_grads_nme|status_y10_grads_nme|
+--------------+--

In [ ]:
pseof_df = pseof_df.na.drop(how='any')

In [ ]:
pseof_df.show()
pseof_df.select('cip_level').count()

+--------------+----------+-----------+------------+---------+-------+-----------+-----------------+---------+---------+---------+--------+------------+--------------------+------------+--------------------+-------------+---------------------+------------+------------+-------------+-------------------+---------------------------+-------------------+---------------------------+--------------------+----------------------------+-------------------+-------------------+--------------------+
|agg_level_pseo|inst_level|institution|degree_level|cip_level|cipcode|grad_cohort|grad_cohort_years|geo_level|geography|ind_level|industry|y1_grads_emp|y1_grads_emp_instate|y5_grads_emp|y5_grads_emp_instate|y10_grads_emp|y10_grads_emp_instate|y1_grads_nme|y5_grads_nme|y10_grads_nme|status_y1_grads_emp|status_y1_grads_emp_instate|status_y5_grads_emp|status_y5_grads_emp_instate|status_y10_grads_emp|status_y10_grads_emp_instate|status_y1_grads_nme|status_y5_grads_nme|status_y10_grads_nme|
+--------------+--

KeyboardInterrupt: ignored

In [ ]:
pseoe_df = pseoe_df.na.drop(how='any')
pseoe_df.show()
pseoe_df.select('cip_level').count()

In [ ]:
pseof_df.columns

In [ ]:
from pyspark.sql.functions import col
cols_to_be_converted_to_int = ['agg_level_pseo',
 'institution',
 'degree_level',
 'cipcode',
 'grad_cohort',
 'grad_cohort_years',
 'geography',
 'industry',
 'y1_grads_emp',
 'y1_grads_emp_instate',
 'y5_grads_emp',
 'y5_grads_emp_instate',
 'y10_grads_emp',
 'y10_grads_emp_instate',
 'y1_grads_nme',
 'y5_grads_nme',
 'y10_grads_nme',
 'status_y1_grads_emp',
 'status_y1_grads_emp_instate',
 'status_y5_grads_emp',
 'status_y5_grads_emp_instate',
 'status_y10_grads_emp',
 'status_y10_grads_emp_instate',
 'status_y1_grads_nme',
 'status_y5_grads_nme',
 'status_y10_grads_nme']
test = pseof_df
for column in cols_to_be_converted_to_int:
  test = test.withColumn(column, col(column).cast('int'))
pseof_df = test

In [ ]:
pseof_df.printSchema()

In [ ]:
pseoe_df.columns

In [ ]:
cols_to_be_converted_to_int = ['agg_level_pseo',
 'institution',
 'degree_level',
 'cipcode',
 'grad_cohort',
 'grad_cohort_years',
 'geography',
 'industry',
 'y1_p25_earnings',
 'y1_p50_earnings',
 'y1_p75_earnings',
 'y1_grads_earn',
 'y5_p25_earnings',
 'y5_p50_earnings',
 'y5_p75_earnings',
 'y5_grads_earn',
 'y10_p25_earnings',
 'y10_p50_earnings',
 'y10_p75_earnings',
 'y10_grads_earn',
 'y1_ipeds_count',
 'y5_ipeds_count',
 'y10_ipeds_count',
 'status_y1_earnings',
 'status_y1_grads_earn',
 'status_y5_earnings',
 'status_y5_grads_earn',
 'status_y10_earnings',
 'status_y10_grads_earn',
 'status_y1_ipeds_count',
 'status_y5_ipeds_count',
 'status_y10_ipeds_count']
test = pseoe_df
for column in cols_to_be_converted_to_int:
  test = test.withColumn(column, col(column).cast('int'))
pseoe_df = test

In [ ]:
pseoe_df.printSchema()

In [3]:
from getpass import getpass
mode = "append"
password = getpass('Enter Password')
jdbc_url="jdbc:postgresql://database-1-instance-1.cfqtepfdzy8v.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter Password··········


In [ ]:
pseof_df.write.jdbc(url=jdbc_url,  table='pseof', mode=mode, properties=config)

In [ ]:
pseoe_df.write.jdbc(url=jdbc_url, table='pseoe', mode=mode, properties=config)

In [4]:
jdbcUrl = 'jdbc:postgresql://database-1-instance-1.cfqtepfdzy8v.us-east-2.rds.amazonaws.com:5432/postgres'
connectionProperties = {"user":"postgres", 
          "password": 'postgres', 
          "driver":"org.postgresql.Driver"}

In [5]:
pushdown_query = "(select * from pseoe)AS pseoe_alias"
df = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)

In [6]:
df.show(20)

+--------------+----------+-----------+------------+---------+-------+-----------+-----------------+---------+---------+---------+--------+---------------+---------------+---------------+-------------+---------------+---------------+---------------+-------------+----------------+----------------+----------------+--------------+--------------+--------------+---------------+------------------+--------------------+------------------+--------------------+-------------------+---------------------+---------------------+---------------------+----------------------+
|agg_level_pseo|inst_level|institution|degree_level|cip_level|cipcode|grad_cohort|grad_cohort_years|geo_level|geography|ind_level|industry|y1_p25_earnings|y1_p50_earnings|y1_p75_earnings|y1_grads_earn|y5_p25_earnings|y5_p50_earnings|y5_p75_earnings|y5_grads_earn|y10_p25_earnings|y10_p50_earnings|y10_p75_earnings|y10_grads_earn|y1_ipeds_count|y5_ipeds_count|y10_ipeds_count|status_y1_earnings|status_y1_grads_earn|status_y5_earnings|s

In [7]:
pseoe_pd = df.toPandas()

In [8]:
import pandas as pd

In [9]:
pseoe_pd

,agg_level_pseo,inst_level,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geo_level,geography,...,y10_ipeds_count,status_y1_earnings,status_y1_grads_earn,status_y5_earnings,status_y5_grads_earn,status_y10_earnings,status_y10_grads_earn,status_y1_ipeds_count,status_y5_ipeds_count,status_y10_ipeds_count
0,38,I,105100,5,A,0,0,3,N,0,...,28151,1,1,1,1,1,1,4,4,4
1,38,I,105100,7,A,0,0,5,N,0,...,5641,1,1,1,1,1,1,1,1,1
2,38,I,105100,17,A,0,0,5,N,0,...,813,1,1,1,1,1,1,1,1,1
3,38,I,105100,18,A,0,0,5,N,0,...,863,1,1,1,1,1,1,1,1,1
4,38,I,105500,5,A,0,0,3,N,0,...,5229,1,1,1,1,1,1,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83913,48,I,182500,5,4,51,2004,3,N,0,...,344,1,1,1,1,1,1,1,1,1
83914,48,I,182500,5,4,52,2004,3,N,0,...,2135,1,1,1,1,1,1,1,1,1
83915,48,I,182500,5,4,52,2004,3,N,0,...,396,1,1,1,1,1,1,1,1,1
83916,48,I,182500,5,4,52,2004,3,N,0,...,450,1,1,1,1,1,1,1,1,1


In [10]:
pushdown_query = "(select * from pseof)AS pseoe_alias"
pseof_df = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties).toPandas()
pseof_df

,agg_level_pseo,inst_level,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geo_level,geography,...,y10_grads_nme,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate,status_y10_grads_emp,status_y10_grads_emp_instate,status_y1_grads_nme,status_y5_grads_nme,status_y10_grads_nme
0,38,I,105100.0,5,A,0,0,3,N,0,...,7281,1,1,1,1,1,1,1,1,1
1,38,I,105100.0,7,A,0,0,5,N,0,...,1149,1,1,1,1,1,1,1,1,1
2,38,I,105100.0,17,A,0,0,5,N,0,...,191,1,1,1,1,1,1,1,1,1
3,38,I,105100.0,18,A,0,0,5,N,0,...,256,1,1,1,1,1,1,1,1,1
4,38,I,105500.0,5,A,0,0,3,N,0,...,993,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56524,46,I,3730300.0,3,2,52,2001,5,N,0,...,25,1,1,1,1,1,1,1,1,1
56525,46,I,3789400.0,3,2,24,2001,5,N,0,...,40,1,1,1,1,1,1,1,1,1
56526,46,I,3956300.0,3,2,13,2001,5,N,0,...,27,1,1,1,1,1,1,1,1,1
56527,46,I,4229500.0,5,2,52,2007,3,N,0,...,18,1,1,1,1,1,1,1,1,1


In [11]:
pushdown_query = "(select * from pseo_all)AS pseoe_alias"
all_df = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties).toPandas()
all_df

,agg_level_pseo,inst_level,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geo_level,geography,...,status_y1_earnings,status_y5_earnings,y1_grads_emp,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate
0,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
1,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
2,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
3,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
4,38,I,105100,5,A,0,0,3,N,0,...,1,1,49897.0,27741.0,32189.0,18740.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223155,48,I,3730300,3,4,52,2001,5,N,0,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223156,48,I,3789400,3,4,24,2001,5,N,0,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223157,48,I,3789400,3,4,24,2001,5,N,0,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223158,48,I,3956300,3,4,13,2001,5,N,0,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
